### (1) Import the necessary packages

In [1]:
import csv
import numpy as np

### (2) Read Intel.csv and Microsoft.csv. Note that they must be in the same directory as this Jupyter notebook.

In [2]:
# initialize the "intel_data" dictionary
intel_data = {"days": [],         # days
              "SVM": [],          # support vector machine prediction
              "RF": [],           # random forest prediction
              "KNN": [],          # k-nearest neighbor prediction
              "ARIMA": [],        # autoregressive integrated moving average prediction
              "MA": [],           # moving average prediction
              "actual": []}       # actual closing price

# read the data from Intel.csv and store it in the dictionary called intel_data
with open('Intel.csv', newline='') as f:
    reader = csv.reader(f)
    for idx, row in enumerate(reader):
        if idx > 0:
            intel_data['days'].append(int(row[0]))
            intel_data['actual'].append(float(row[1]))
            intel_data['RF'].append(float(row[2]))
            intel_data['KNN'].append(float(row[3]))
            intel_data['SVM'].append(float(row[4]))
            intel_data['ARIMA'].append(float(row[5]))
            intel_data['MA'].append(float(row[6]))

# print the "intel_data" dictionary
#print(f"intel_data = {intel_data}")

In [3]:
# initialize the "microsoft_data" dictionary
microsoft_data = {"days": [],         # days
                  "SVM": [],          # support vector machine prediction
                  "RF": [],           # random forest prediction
                  "KNN": [],          # k-nearest neighbor prediction
                  "ARIMA": [],        # autoregressive integrated moving average prediction
                  "MA": [],           # moving average prediction
                  "actual": []}       # actual closing price

# read the data from Microsoft.csv and store it in the dictionary called microsoft_data
with open('Microsoft.csv', newline='') as f:
    reader = csv.reader(f)
    for idx, row in enumerate(reader):
        if idx > 0:
            microsoft_data['days'].append(int(row[0]))
            microsoft_data['actual'].append(float(row[1]))
            microsoft_data['RF'].append(float(row[2]))
            microsoft_data['KNN'].append(float(row[3]))
            microsoft_data['SVM'].append(float(row[4]))
            microsoft_data['ARIMA'].append(float(row[5]))
            microsoft_data['MA'].append(float(row[6]))

# print the "microsoft_data" dictionary
#print(f"microsoft_data = {microsoft_data}")

### (3) Define the loss function

In [4]:
def loss(prediction, actual):
    # prediction: prediction value
    # actual: actual value
    
    return min(np.abs(prediction - actual) / actual, 5)

### (4) Implement the multiplicative weight update algorithm (MWU) on Intel stock prices

In [23]:
# Input Parameters
n = 5  # number of experts
T = 150  # time horizon
eps =  0.1 # parameter epsilon  

from random import random
from random import uniform

# Initialization
w_exp = [1,1,1,1,1]  # numpy array of the experts' weights
l_exp = [0,0,0,0,0]  # numpy array of the total losses of experts
loss_exp = [0,0,0,0,0]  # numpy array of loss of each experts
l_mwu = 0  # the total loss of MWU

# Run the multiplicative weight update algorithm
for t in range(1, T+1):
    # get experts' predictions for day t from the intel_data dictionary in (2)
    predRF = intel_data['RF'][t-1]
    predKNN = intel_data['KNN'][t-1]
    predSVM = intel_data['SVM'][t-1]
    predARIMA = intel_data['ARIMA'][t-1]
    predMA = intel_data['MA'][t-1]
    exp_preds = [predRF, predKNN, predSVM, predARIMA, predMA]
    #print("exp_preds", exp_preds)
    
    # predict the closing price for day t according to MWU
    choice = random()*sum(w_exp)  # https://jeremykun.com/tag/multiplicative-weights-update-algorithm/
    outcome = 0
    for weight in w_exp:
        choice -= weight
        if choice <= 0:
            break
        else: outcome = outcome + 1
    #print('Predicted Outcome: ', exp_preds[outcome])
    
    # get the true closing price for day t from the intel_data dictionary in (2)
    actual = intel_data['actual'][t-1]
    #print("Actual Outcome: ", actual)
    
    # update the experts' weights (w_exp) according to MWU
    for i in range(n):
        if (exp_preds[i] != actual):
            w_exp[i] *= 1-eps*loss(exp_preds[i], actual)
    
    # update the total losses incurred by experts (l_exp) and MWU (l_mwu)
    l = loss(outcome, actual)
    l_mwu += l 
    l_exp[outcome] += 1
    loss_exp[outcome] += l
    #print("Total losses incurred by experts: ", l_exp)
    
    # print the experts' weights (w_exp) after round t
    print("Current experts' weights for day ", t, ": ", w_exp)

    
# print the total loss incurred by MWU over T time periods
print("Total losses incurred by WMA: ", l_mwu)

# print the total loss incurred by the best expert from hindsight
bestindex = np.argmax(w_exp)
nummax = loss_exp[bestindex]
print("Total loss incurred by the best expert from hindsight: ", nummax)

# print the regret of MWU based on this single instance of stock prices
regret = l_mwu - nummax
print("Regret = ", regret)

upperbest = l_mwu*np.log(n)/eps + nummax + nummax*eps
print("Upper bound of mistakes with best expert: ", upperbest) 


Current experts' weights for day  1 :  [0.9973072196701802, 0.9897350268446107, 0.9970759894253655, 0.9994691707412445, 0.9964875878952737]
Current experts' weights for day  2 :  [0.9946227245839067, 0.9798643778024086, 0.9948383376779992, 0.9990742339944403, 0.9939650074957774]
Current experts' weights for day  3 :  [0.9920443469294652, 0.9700976770518838, 0.9931358686064287, 0.9988491301870122, 0.9923603078938854]
Current experts' weights for day  4 :  [0.9913345234208161, 0.9634642930063168, 0.992985283646, 0.9970846951833204, 0.9912199820502224]
Current experts' weights for day  5 :  [0.9822430301476753, 0.9630600164243907, 0.9866130045261292, 0.985222318127281, 0.9794153317304559]
Current experts' weights for day  6 :  [0.9788130096870966, 0.9586050173434684, 0.980214584448635, 0.9705768112338891, 0.9652846173436292]
Current experts' weights for day  7 :  [0.9781159521582171, 0.9552619174195984, 0.9726652248715685, 0.9559201143605242, 0.9534984636500629]
Current experts' weights f

### (5) Implement the multiplicative weight update algorithm (MWU) on Microsoft stock prices

In [22]:
# Input Parameters
n = 5  # number of experts
T = 150  # time horizon
eps =  0.1 # parameter epsilon 

from random import random
from random import uniform

# Initialization
w_exp = [1,1,1,1,1]  # numpy array of the experts' weights
l_exp = [0,0,0,0,0]  # numpy array of the total losses of experts
loss_exp = [0,0,0,0,0]  # numpy array of loss of each experts
l_mwu = 0  # the total loss of MWU

# Run the multiplicative weight update algorithm
for t in range(1, T+1):
    # get experts' predictions for day t from the intel_data dictionary in (2)
    predRF = microsoft_data['RF'][t-1]
    predKNN = microsoft_data['KNN'][t-1]
    predSVM = microsoft_data['SVM'][t-1]
    predARIMA = microsoft_data['ARIMA'][t-1]
    predMA = microsoft_data['MA'][t-1]
    exp_preds = [predRF, predKNN, predSVM, predARIMA, predMA]
    #print("exp_preds", exp_preds)
    
    # predict the closing price for day t according to MWU
    choice = random()*sum(w_exp)  # https://jeremykun.com/tag/multiplicative-weights-update-algorithm/
    outcome = 0
    for weight in w_exp:
        choice -= weight
        if choice <= 0:
            break
        else: outcome += 1
    #print('Predicted Outcome: ', exp_preds[outcome])
    
    # get the true closing price for day t from the intel_data dictionary in (2)
    actual = microsoft_data['actual'][t-1]
    #print("Actual Outcome: ", actual)
    
    # update the experts' weights (w_exp) according to MWU
    for i in range(n):
        if (exp_preds[i] != actual):
            w_exp[i] *= 1-eps*loss(exp_preds[i], actual)
    
    # update the total losses incurred by experts (l_exp) and MWU (l_mwu)
    l = loss(outcome, actual)
    l_mwu += l 
    l_exp[outcome] += 1
    loss_exp[outcome] += l
    #print("Total losses incurred by experts: ", l_exp)
    
    # print the experts' weights (w_exp) after round t
    print("Current experts' weights for day ", t, ": ", w_exp)

    
# print the total loss incurred by MWU over T time periods
print("Total losses incurred by WMA: ", l_mwu)

# print the total loss incurred by the best expert from hindsight
bestindex = np.argmax(w_exp)
nummax = loss_exp[bestindex]
print("Total loss incurred by the best expert from hindsight: ", nummax)

# print the regret of MWU based on this single instance of stock prices
regret = l_mwu - nummax
print("Regret = ", regret)

upperbest = l_mwu*np.log(n)/eps + nummax + nummax*eps
print("Upper bound of mistakes with best expert: ", upperbest) 


Current experts' weights for day  1 :  [0.9974792431726044, 0.9526826114627891, 0.9510142463313793, 0.9993742199519356, 0.9977488665522161]
Current experts' weights for day  2 :  [0.993780965445378, 0.9060547815014, 0.9045075848219023, 0.9974005212279897, 0.9946802140809688]
Current experts' weights for day  3 :  [0.9902096915368973, 0.8618174757855503, 0.8624855216085043, 0.9958654234746621, 0.9923997924831478]
Current experts' weights for day  4 :  [0.9834230214795241, 0.8190268875122915, 0.8212734733192445, 0.9910486175922714, 0.9876909887216409]
Current experts' weights for day  5 :  [0.9761609929875708, 0.7794469982791362, 0.7869842564461946, 0.9856956766251473, 0.9831133479294625]
Current experts' weights for day  6 :  [0.9692011241165702, 0.7419071673329103, 0.7545292555407994, 0.980608209415692, 0.9797618054622022]
Current experts' weights for day  7 :  [0.9618725262571296, 0.7053713620294564, 0.7233096706954252, 0.9749313464389607, 0.9767985091158766]
Current experts' weights 